# Webscraping utilizando BeautifulSoup e Selenium
# Busca de produtos OLX




In [3]:
# !pip install bs4

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import time

#selenium
#!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [ ]:
#pg1: https://sp.olx.com.br/regiao-de-bauru-e-marilia?ot=1&q=honda
#pg2: https://sp.olx.com.br/regiao-de-bauru-e-marilia?ot=1&o=2&q=honda
# trocar pagina = &o=2

# Last page: <a href="https://sp.olx.com.br/regiao-de-bauru-e-marilia?ot=1&amp;o=12&amp;q=honda" data-lurker-detail="last_page" class="sc-1bofr6e-0 iRQkdN"><span class="sc-1bofr6e-1 iUNkan sc-ifAKCX bBbnjQ" color="dark" font-weight="400">Última pagina</span></a>

# numero de paginas o=12
# data-lurker-detail="last_page" 
# class="sc-1bofr6e-0 iRQkdN">
# 


     |████████████████████████████████| 115 kB 2.9 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=56b9db4c7aad64f32f9b7be46b87a1fbd14cf5cc55277a8da6c755e3f2f3f628
  Stored in directory: /home/cristian/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


# Webscraping do site OLX - Busca de produtos


In [4]:
# Buscar:
# Link da busca do produto
url = "https://sp.olx.com.br/regiao-de-bauru-e-marilia?ot=1&q=honda"

# Necessário passar alguns parametros  para que o request não seja bloqueado pelo site.
PARAMS = {
    #definicoes do header do site
    "autority": "sp.olx.com.br",
    "method": "GET",
    "path": "/regiao-de-bauru-e-marilia",
    "scheme": "https",
    "referrer": "https://sp.olx.com.br/",
    "sec-fetch-mode": "navigate",
    "sec-fecth-site": "same-origin",
    "sec-fech-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0"
}

# Ler o conteudo da pagina e salvar numa variavel
pages = requests.get(url = url, headers = PARAMS)

# Converter o conteudo da pagina no formato lxml, que possibilita buscar partes da pagina utilizando tags, classes, etc.
conteudo = BeautifulSoup(pages.content, "lxml")

#results = conteudo.find_all("li", {"class" : "nome_classe"}) # resultado da busca - uma lista 

#for a in results: # percorrer todos os resultados da busca
#    try:
#        a.find_all("h2")[0].contents[0]) # retorna o titulo do anuncio
#        a.find_all("p", {"class" : "nome_classe"})[0].contents[0] # retorna a informação de um objeto/classe
#    except:
#        print(erro)

#print(conteudo)


In [5]:
print(pages)

<Response [200]>


In [ ]:
#print(conteudo.find_all("div", {"class":"fnmrjs-9 gqfQzY"}))

#print(len(conteudo.find_all("h2")))

100


Este exemplo está retornando apenas os resultados da primeira página da busca.


Para obter o resultado das outras páginas é necessario utilizar um loop, adicionando ao link da busca &o=2 (numero da pagina).

In [8]:
# Busca na página por uma lista, que nesse exmplo são os produtos anunciados e salva numa variavel itens
# Para obter as tags como "li" e a "class" utilizamos o inspect do site
itens = conteudo.find_all("li", {"class" : "sc-1fcmfeb-2 juiJqh"})


# Passar pela lista dos itens para obter detalhes dos anuncios
for a in itens:
    try:
        nomeItem = a.find_all("h2")[0].contents[0] 
        linkItem = a.find("a")["href"]
        precoItem = a.find_all("span", {"class" : "sc-ifAKCX eoKYee"})[0].contents[0]
        precoItem = (precoItem.split()[1])
        precoItem = float(precoItem.replace(".", ""))
        precoOferta = precoItem * 0.8
        # <span color="dark" class="wlwg1t-1 fsgKJO sc-ifAKCX eLPYJb" font-weight="400">Hoje</span>
        data = a.find_all("span", {"class" : "wlwg1t-1 fsgKJO sc-ifAKCX eLPYJb"})
        dataItem = data[0].contents[0]
        horaItem = data[1].contents[0]
        horaItem = dataItem[1]
        #<span color="dark" class="wlwg1t-1 fsgKJO sc-ifAKCX eLPYJb" font-weight="400">16:31</span>
        horaItem = a.find_all("span", {"class" : "wlwg1t-1 fsgKJO sc-ifAKCX eLPYJb"})
        print("Link: ", linkItem)
        print("Preço: ",precoItem)
        print("Oferta: ", precoOferta)
        print("------")
        print(nomeItem, precoItem)
        if dataItem == "Hoje":
            print(dataItem, horaItem)
                                       
        

    except:
        print("")


        
        
#print(linkItem)



Link:  https://sp.olx.com.br/regiao-de-bauru-e-marilia/ciclismo/skate-hondar-904571455
Preço:  1.0
Oferta:  0.8
------
Skate Hondar 1.0


Link:  https://sp.olx.com.br/regiao-de-bauru-e-marilia/autos-e-pecas/carros-vans-e-utilitarios/honda-civic-904458664
Preço:  20000.0
Oferta:  16000.0
------
Honda Civic. 20000.0
Link:  https://sp.olx.com.br/regiao-de-bauru-e-marilia/autos-e-pecas/carros-vans-e-utilitarios/honda-civic-904723723
Preço:  36500.0
Oferta:  29200.0
------
Honda Civic  36500.0
Hoje [<span class="wlwg1t-1 fsgKJO sc-ifAKCX eLPYJb" color="dark" font-weight="400">Hoje</span>, <span class="wlwg1t-1 fsgKJO sc-ifAKCX eLPYJb" color="dark" font-weight="400">11:40</span>]
Link:  https://sp.olx.com.br/regiao-de-bauru-e-marilia/autos-e-pecas/motos/honda-150-904363143
Preço:  7000.0
Oferta:  5600.0
------
Honda 150  7000.0

Link:  https://sp.olx.com.br/regiao-de-bauru-e-marilia/autos-e-pecas/pecas-e-acessorios/motos/escapamento-honda-904139228
Preço:  150.0
Oferta:  120.0
------
Escapam

In [9]:
print(linkItem)

https://sp.olx.com.br/regiao-de-bauru-e-marilia/autos-e-pecas/motos/honda-xre-300-p-903299041


# Utilizar o Selenium para navegar automaticamente pelo site

In [ ]:
#!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [ ]:
# Necessário instalar um pacote/driver do sistema operacional
# $ sudo apt install firefox-geckodriver

# Inicializar o driver do Firefox:
browser = webdriver.Firefox()

# Abrir site de login:
browser.get("https://conta.olx.com.br/acesso/")

# fazer login - Utilizamos o inspect para descobrir o type dos campos que precisam ser preenchidos
username = browser.find_element_by_xpath("//input[@type='email']")
password = browser.find_element_by_xpath("//input[@type='password']")

username.send_keys("usuario")
password.send_keys("senha")

# Localizar o botão de submit e clicar no botão para realizar o login
browser.find_element_by_xpath("//button[@type='text']").click()

# Aguardar alguns segundos para realizar o login com sucesso
time.sleep(5)

# Abrir link da nova pagina apos fazer login
browser.get(linkItem)

# Clicar no botao Chat
#<span color="white" data-testid="largeFloatButton-title" font-weight="400" class="sc-ifAKCX eUXUWN">Chat</span>
browser.find_element_by_xpath("//span[@data-testid='largeFloatButton-title']").click()
time.sleep(5)


# inserir mensagem no campo do Chat:
#<textarea placeholder="Digite uma mensagem..." class="sc-hycgNl gACIxP sc-chAAoq fEgJLD" style="height: 47px;"></textarea>
chat = browser.find_element_by_xpath("//textarea[@placeholder='Digite uma mensagem...']")
chat.send_keys("Aceita troca?" + Keys.ENTER)

# Fecha o navegador
browser.quit()